<a href="https://colab.research.google.com/github/fevdaele/Machine_Learning_course_UGent_D012554_kaggle/blob/master/PredictionsFemke_12_05_2020_version3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt;
import numpy as np;
import pandas as pd;
import seaborn as sns;
sns.set_context("notebook", font_scale=1.4);
sns.set_style("whitegrid");
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

### **Load train and dataset**

In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")

testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")

### **Data pre processing**

In [0]:
data_train = trainset.copy()
label_column = data_train.pop('label')

data_test = testset.copy()
index_column = data_test.pop('index')

**Scaling of trainset**

In [0]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_train = scaler.fit_transform(data_train)


#plt.figure(figsize=(12,8))
#trainset_scaled.boxplot(vert=False)
#plt.show()

**Scaling of testset**

In [0]:
data_test = scaler.transform(data_test)
#plt.figure(figsize=(12,8))
#testset_scaled.boxplot(vert=False)
#plt.show()


**Feature extraction**

In [55]:
from sklearn.decomposition import PCA

pca = PCA(n_components=7)
pca.fit_transform(data_train)
print(pca.explained_variance_ratio_.sum())


0.9468205971550513


In [0]:
pca = PCA(n_components=7)
data_train = pd.DataFrame(pca.fit(data_train).transform(data_train),
                                     columns=['PC1','PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7'])
data_train['label'] = label_column

data_test = pd.DataFrame(pca.transform(data_test), columns=['PC1','PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7'])
data_test['index'] = index_column


### ***Model selection and validation***

In [57]:
from sklearn.svm import SVC

y = label_column
X = data_train

model = SVC(kernel='rbf', gamma=1 , C=1)

score_auc = cross_val_score(model, data_train, y=trainset['label'], scoring = 'roc_auc', cv=10).mean()
print(score_auc)


0.9998383838383837


### ***Model optimization***

In [58]:
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn import metrics

X = data_train
y = label_column


search_space_C = [0.001, 0.01, 0.1, 1, 10]
search_space_gamma = [1, 2, 3, 4, 5, 6]
param_grid = {'gamma': search_space_gamma, 'C' : search_space_C}
grid_search = GridSearchCV(model, param_grid, scoring = 'roc_auc', cv=10)
grid_search.fit(X, y)

print(grid_search.best_estimator_)
print(grid_search.best_params_)
print(grid_search.best_score_)


SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)
{'C': 1, 'gamma': 1}
0.9998383838383837


**Define and validate model with optimized hyper parameter**

In [59]:
model = SVC(kernel='rbf', gamma=1, C=1, probability=True)
score_auc = cross_val_score(model, data_train, y=trainset['label'], scoring = 'roc_auc', cv=10).mean()
print(score_auc)

0.9998383838383837


### ***Fit model on trainset***

In [60]:
model.fit(X,y)

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

### ***Make predictions***

In [64]:

predictions = model.predict_proba(data_test)
#predictions = model.predict_proba(data_test)
print(predictions)

[[9.99999535e-01 4.65354768e-07]
 [5.44753433e-14 1.00000000e+00]
 [5.95888035e-03 9.94041120e-01]
 ...
 [5.59199341e-01 4.40800659e-01]
 [5.59199341e-01 4.40800659e-01]
 [5.59199341e-01 4.40800659e-01]]


**Create new dataframe to load predictions**

In [65]:
PredictionsFemke = pd.DataFrame(columns=['index','label'])
PredictionsFemke['index'] = index_column
PredictionsFemke['label'] = predictions[:,1]

print(PredictionsFemke)

       index         label
0          0  4.653548e-07
1          1  1.000000e+00
2          2  9.940411e-01
3          3  5.315494e-01
4          4  4.413153e-01
...      ...           ...
12887  12887  4.408007e-01
12888  12888  4.408007e-01
12889  12889  4.408007e-01
12890  12890  4.408007e-01
12891  12891  4.408007e-01

[12892 rows x 2 columns]


### **Save predictions in csv file**

In [0]:
filename = "PredictionsFemke12052020_version3.csv"

#make sure to not write the Pandas index column (index=False)
PredictionsFemke.to_csv(filename,index=False)